In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as obj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
train = pd.read_json("train.json")

In [30]:
test = pd.read_json("test.json")

In [3]:
train['band_1'][0][0:5]

[-27.878360999999998, -27.15416, -28.668615, -29.537971, -29.092485]

In [4]:
len(train['band_1'][0])

5625

In [5]:
test.head(n=4)

NameError: name 'test' is not defined

In [6]:
train.head(n=15)

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0
5,"[-20.769371, -20.769434, -25.906025, -25.90602...","[-29.288746, -29.712593, -28.884804, -28.88480...",b51d18b5,36.9034,1
6,"[-26.673811, -23.666162, -27.622442, -28.31768...","[-24.557735, -26.97868, -27.622442, -29.073456...",31da1a04,34.4751,1
7,"[-24.989119, -27.755224, -25.817074, -24.98927...","[-27.755173, -26.732174, -28.124943, -31.83772...",56929c16,41.1769,0
8,"[-17.146641, -17.146572, -17.994583, -19.44553...","[-25.733608, -24.472507, -24.710424, -22.77215...",525ab75c,35.7829,0
9,"[-24.020853, -23.551275, -27.18819, -29.126434...","[-28.702518, -33.563324, -29.571918, -29.12643...",192f56eb,43.3007,0


In [7]:
#Generate the training data
#Create 3 bands having HH, HV and avg of both
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [8]:
#Take a look at a iceberg
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):

    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)


In [9]:
plotmy3d(X_band_1[10,:,:], 'iceberg') #5 6 10 12 13

In [10]:
plotmy3d(X_band_1[8,:,:], 'notanIceberg') #anything except 5 6 10 12 13 why is 8 so funny

In [11]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.layers.advanced_activations import LeakyReLU ## I am adding this just to see
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam, Nadam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [12]:
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='linear', input_shape=(75, 75, 3))) ## removed relu and added linear
    gmodel.add(LeakyReLU(alpha=.001))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='linear' ))
    gmodel.add(LeakyReLU(alpha=.001))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='linear'))
    gmodel.add(LeakyReLU(alpha=.001))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='linear'))
    gmodel.add(LeakyReLU(alpha=.001))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    #mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    mypotim = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=3):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=False)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

In [13]:
target_train=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.80)

In [27]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.4, min_lr=0.00001)


In [28]:
#Without denoising, core features.
import os
gmodel=getModel()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 73, 73, 64)        1792      
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 73, 73, 64)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 34, 34, 128)       0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 128)       0         
__________

In [29]:
gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=120,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks = [learning_rate_reduction])

Train on 1283 samples, validate on 321 samples
Epoch 1/120
1283/1283 [==============================] - 4s 3ms/step - loss: 0.8720 - acc: 0.5737 - val_loss: 0.6094 - val_acc: 0.6729
Epoch 2/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.5575 - acc: 0.6687 - val_loss: 0.5676 - val_acc: 0.6791
Epoch 3/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.5486 - acc: 0.6851 - val_loss: 0.5438 - val_acc: 0.6916
Epoch 4/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.5358 - acc: 0.7124 - val_loss: 0.5843 - val_acc: 0.7664
Epoch 5/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.4890 - acc: 0.7685 - val_loss: 0.4365 - val_acc: 0.8193
Epoch 6/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.4617 - acc: 0.7724 - val_loss: 0.3963 - val_acc: 0.8349
Epoch 7/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.4245 - acc: 0.7880 - val_loss: 0.3801 - val_acc: 0.8380
Epoch 8/

Epoch 55/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1244 - acc: 0.9447 - val_loss: 0.2500 - val_acc: 0.9065
Epoch 56/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1261 - acc: 0.9439 - val_loss: 0.2481 - val_acc: 0.9034
Epoch 57/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1252 - acc: 0.9431 - val_loss: 0.2475 - val_acc: 0.9097
Epoch 58/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1298 - acc: 0.9408 - val_loss: 0.2488 - val_acc: 0.9065
Epoch 59/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1259 - acc: 0.9501 - val_loss: 0.2469 - val_acc: 0.9034
Epoch 60/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1251 - acc: 0.9470 - val_loss: 0.2451 - val_acc: 0.9034
Epoch 61/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1281 - acc: 0.9470 - val_loss: 0.2451 - val_acc: 0.9034
Epoch 62/120
1283/1283 [========================

1283/1283 [==============================] - 3s 2ms/step - loss: 0.1122 - acc: 0.9532 - val_loss: 0.2699 - val_acc: 0.9034
Epoch 116/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1191 - acc: 0.9509 - val_loss: 0.2650 - val_acc: 0.9097
Epoch 117/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1211 - acc: 0.9470 - val_loss: 0.2622 - val_acc: 0.9097
Epoch 118/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1071 - acc: 0.9556 - val_loss: 0.2615 - val_acc: 0.9065
Epoch 119/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1047 - acc: 0.9579 - val_loss: 0.2641 - val_acc: 0.9065
Epoch 120/120
1283/1283 [==============================] - 3s 2ms/step - loss: 0.1145 - acc: 0.9517 - val_loss: 0.2595 - val_acc: 0.9097


In [31]:
gmodel.load_weights(filepath=file_path)
score = gmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

321/321 [==============================] - 1s 2ms/step
Test loss: 0.39690263704
Test accuracy: 0.822429906542


In [35]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
predicted_test=gmodel.predict_proba(X_test)

In [36]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

In [ ]:
#comment everything else
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

gmodel.fit_generator(datagen.flow(X_train_cv,y_train_cv, batch_size=24),
                              epochs = 100, validation_data = (X_valid,y_valid)
                              , callbacks=[learning_rate_reduction])